In [64]:
class Point:
    """ A class that represents a geographical point
        which consists of an X, Y coordinate
    """
    
    def __init__(self, lat: float = 0, lon:float = 0 ):
        self.__x = lon
        self.__y = lat
        
    def __repr__(self):
        return f'Point(lon={self.__x}, lat={self.__y})'
    
    def __str__(self):
        return f'{self.__x},{self.__y}'
    
    def __gt__(self, other):
        return self.lat > other.lat and self.lon > other.lon
    
    def __lt__(self, other):
        return self.lat < other.lat and self.lon < other.lon
    
    def __eq__(self, other):
        return self.lat == other.lat and self.lon == other.lon
    
    @property
    def lon(self):
        return self.__x
    
    @property
    def lat(self):
        return self.__y
    
    @property
    def as_tuple(self):
        return (self.lat, self.lon)

In [15]:
def haversine_distance(origin:Point, destination:Point) -> float:
    """Calculates the Haversine distance between
        two points (x, y)
    """
    
    import math
    
    EARTH_RADIUS = 6371000
    
    diff_lat = math.radians(destination.lat - origin.lat)
    diff_lon = math.radians(destination.lon - origin.lon)
    
    a = math.sin(diff_lat / 2) * math.sin(diff_lat / 2) + math.cos(math.radians(origin.lat)) \
        * math.cos(math.radians(destination.lat)) * math.sin(diff_lon / 2) * math.sin(diff_lon / 2)
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    distance = EARTH_RADIUS * c
    return distance
    
    

In [65]:
san_francisco = Point(37.7749, -122.4194)
new_york = Point(40.661, -73.944)

In [36]:
print(san_francisco)

(-122.4194, 37.7749)


In [37]:
new_york

Point(lon=-73.944, lat=40.661)

In [43]:
distance_ = haversine_distance(san_francisco, new_york)
print(distance_ / 1000, 'km')

4135.374617164737 km


In [39]:
print(new_york > san_francisco)

True


In [40]:
print(new_york == san_francisco)

False


In [41]:
san_francisco.as_tuple

(37.7749, -122.4194)

In [53]:
str(san_francisco)

'-122.4194,37.7749'

In [45]:
from geopy import distance as dt

In [52]:
sf = san_francisco.as_tuple
ny = new_york.as_tuple

# Haversine formula
straigh_line_distance = dt.great_circle(sf, ny)
# Vincenty's formula
ellipsoid_distance = dt.geodesic(sf, ny, ellipsoid='WGS-84')

print(straigh_line_distance, ellipsoid_distance)

4135.3804590061345 km 4145.446977549561 km


#### Calculating Routes using the OpenRouteService API

In [48]:
import requests

response = requests.get('http://spatialthoughts.com')
print(response.status_code)

200


In [49]:
OPEN_ROUTE_SERVICE_KEY = '5b3ce3597851110001cf62482792ba47f66d4d00bfe816b84e9a9dae'

In [55]:
parameters = {
    'api_key': OPEN_ROUTE_SERVICE_KEY,
    'start': str(san_francisco),
    'end': str(new_york)
}

response = requests.get('https://api.openrouteservice.org/v2/directions/driving-car', params=parameters)

if response.status_code == 200:
    print('Request successful')
    data = response.json()
else:
    print('Request failed')

Request successful


In [62]:
summary = data['features'][0]['properties']['summary']
print(summary['distance'] / 1000)

4692.2778


In [66]:
def  get_driving_distance(source: Point, destination: Point):
    """ Returns the driving distance between two points """
    
    parameters = {
        'api_key': OPEN_ROUTE_SERVICE_KEY,
        'start': str(source),
        'end': str(destination)
    }
    
    response = requests.get('https://api.openrouteservice.org/v2/directions/driving-car', params=parameters)
    
    if response.status_code == 200:
        data = response.json()
        summary = data['features'][0]['properties']['summary']
        distance = summary['distance'] / 1000
        return distance
        
    else:
        print('Request failed.')
        return -1
    

In [70]:
source = san_francisco

destination_cities = {
    'Los Angeles': Point(34.0522, -118.2437),
    'Boston': Point(42.3601, -71.0589),
    'Atlanta': Point(33.7490, -84.3880)
}

In [79]:
import time
for city_name, dest in destination_cities.items():
    print(get_driving_distance(source, dest))
    time.sleep(2)

615.3162
4986.664900000001
3975.4229


### Working with Pandas

In [80]:
import os
import pandas as pd

data_pkg_path = 'data'
filename = 'worldcities.csv'
path = os.path.join(data_pkg_path, filename)
df = pd.read_csv(path)
print(df.head())

          city   city_ascii      lat       lng        country iso2 iso3  \
0        Tokyo        Tokyo  35.6850  139.7514          Japan   JP  JPN   
1     New York     New York  40.6943  -73.9249  United States   US  USA   
2  Mexico City  Mexico City  19.4424  -99.1310         Mexico   MX  MEX   
3       Mumbai       Mumbai  19.0170   72.8570          India   IN  IND   
4    São Paulo    Sao Paulo -23.5587  -46.6250         Brazil   BR  BRA   

         admin_name  capital  population          id  
0             Tōkyō  primary  35676000.0  1392685764  
1          New York      NaN  19354922.0  1840034016  
2  Ciudad de México  primary  19028000.0  1484247881  
3       Mahārāshtra    admin  18978000.0  1356226629  
4         São Paulo    admin  18845000.0  1076532519  


In [81]:
# Filtering data

filtered = df[df['country'] == 'India']
print(filtered)

           city city_ascii      lat      lng country iso2 iso3  \
3        Mumbai     Mumbai  19.0170  72.8570   India   IN  IND   
5         Delhi      Delhi  28.6700  77.2300   India   IN  IND   
7       Kolkata    Kolkata  22.4950  88.3247   India   IN  IND   
34      Chennai    Chennai  13.0900  80.2800   India   IN  IND   
36    Bengalūru  Bengaluru  12.9700  77.5600   India   IN  IND   
...         ...        ...      ...      ...     ...  ...  ...   
7305      Karūr      Karur  10.9504  78.0833   India   IN  IND   
7441     Jorhāt     Jorhat  26.7500  94.2167   India   IN  IND   
7583      Sopur      Sopur  34.3000  74.4667   India   IN  IND   
7681     Tezpur     Tezpur  26.6338  92.8000   India   IN  IND   
9384        Diu        Diu  20.7197  70.9904   India   IN  IND   

             admin_name capital  population          id  
3           Mahārāshtra   admin  18978000.0  1356226629  
5                 Delhi   admin  15926000.0  1356872604  
7           West Bengal   admin  

In [82]:
country_df = df[df['country'] == 'Colombia'].copy()
filtered = country_df[country_df['city_ascii'] == 'Cali']
print(filtered.iloc[0])

city                     Cali
city_ascii               Cali
lat                       3.4
lng                     -76.5
country              Colombia
iso2                       CO
iso3                      COL
admin_name    Valle del Cauca
capital                 admin
population          2254000.0
id                 1170417589
Name: 178, dtype: object


In [83]:
city_coordinates = (filtered.iloc[0]['lat'], filtered.iloc[0]['lng'])
print(city_coordinates)

(3.4, -76.5)


In [85]:
from geopy import distance
def calculate_distance_to_home_city(row):
    city_coords = (row['lat'], row['lng'])
    return distance.geodesic(city_coords, city_coordinates).km

result = country_df.apply(calculate_distance_to_home_city, axis=1)
print(result)

29       299.207840
100      334.068424
178        0.000000
235      856.878700
491      556.771455
            ...    
10902    177.419668
14272    593.326841
14278    266.915039
14300    434.480165
14326    253.748533
Length: 72, dtype: float64


In [86]:
country_df['distance'] = result
country_df.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id,distance
29,Bogotá,Bogota,4.5964,-74.0833,Colombia,CO,COL,Bogotá,primary,7772000.0,1170483426,299.207840
100,Medellín,Medellin,6.2750,-75.5750,Colombia,CO,COL,Antioquia,admin,3297000.0,1170680389,334.068424
178,Cali,Cali,3.4000,-76.5000,Colombia,CO,COL,Valle del Cauca,admin,2254000.0,1170417589,0.000000
235,Barranquilla,Barranquilla,10.9600,-74.8000,Colombia,CO,COL,Atlántico,admin,1798000.0,1170179113,856.878700
491,Bucaramanga,Bucaramanga,7.1301,-73.1259,Colombia,CO,COL,Santander,admin,1009000.0,1170940590,556.771455


### Geopandas

In [2]:
import os
import pandas as pd
import geopandas as gpd

In [3]:
folder = 'data/geonames'
file = 'US.txt'
file_path = os.path.join(folder, file)

In [4]:
column_names = [
    'geonameid', 'name', 'asciiname', 'alternatenames', 
    'latitude', 'longitude', 'feature class', 'feature code',
    'country code', 'cc2', 'admin1 code', 'admin2 code',
    'admin3 code', 'admin4 code', 'population', 'elevation',
    'dem', 'timezone', 'modification date'
]

df = pd.read_csv(file_path, sep='\t', names=column_names)
print(df.info())

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (9,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2237861 entries, 0 to 2237860
Data columns (total 19 columns):
 #   Column             Dtype  
---  ------             -----  
 0   geonameid          int64  
 1   name               object 
 2   asciiname          object 
 3   alternatenames     object 
 4   latitude           float64
 5   longitude          float64
 6   feature class      object 
 7   feature code       object 
 8   country code       object 
 9   cc2                object 
 10  admin1 code        object 
 11  admin2 code        object 
 12  admin3 code        float64
 13  admin4 code        float64
 14  population         int64  
 15  elevation          float64
 16  dem                int64  
 17  timezone           object 
 18  modification date  object 
dtypes: float64(5), int64(3), object(11)
memory usage: 324.4+ MB
None


In [5]:
df.columns

Index(['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude',
       'longitude', 'feature class', 'feature code', 'country code', 'cc2',
       'admin1 code', 'admin2 code', 'admin3 code', 'admin4 code',
       'population', 'elevation', 'dem', 'timezone', 'modification date'],
      dtype='object')

In [6]:
df.head()

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date
0,2130833,McArthur Reef,McArthur Reef,NaN,52.06667,177.86667,U,RFU,US,NaN,AK,16.0,NaN,NaN,0,NaN,-9999,Asia/Kamchatka,2016-07-05
1,3577483,The Narrows,The Narrows,The Narrows,18.37502,-64.72517,H,CHN,US,VG,00,NaN,NaN,NaN,0,NaN,-9999,America/St_Thomas,2018-11-06
2,3831601,Nantucket Shoals,Nantucket Shoals,NaN,41.06917,-69.67983,U,SHSU,US,NaN,MA,NaN,NaN,NaN,0,NaN,-9999,NaN,2019-03-18
3,3831610,Little Stellwagen Basin,Little Stellwagen Basin,NaN,42.11667,-70.28333,U,BSNU,US,NaN,00,NaN,NaN,NaN,0,NaN,-9999,America/New_York,2017-05-26
4,3831661,Cashes Ledge,Cashes Ledge,"Cash Ledge,Cashe Ledge",42.91328,-68.95380,U,RDGU,US,NaN,NaN,NaN,NaN,NaN,0,NaN,-9999,NaN,2019-02-11


In [10]:
df['feature class'].value_counts()

S    1145159
H     506986
T     225328
P     193111
L      80152
A      55834
R      26263
V       2173
U        198
Name: feature class, dtype: int64

In [11]:
mountains = df[df['feature class'] == 'T']
mountains.head()

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date
15,4045410,Vulcan Point,Vulcan Point,"Volcan Point,Vulcan Point",52.10222,177.53889,T,CAPE,US,NaN,AK,16.0,NaN,NaN,0,NaN,-9999,America/Adak,2014-10-08
16,4045411,Tropical Ridge,Tropical Ridge,NaN,51.99167,177.50833,T,RDGE,US,NaN,AK,16.0,NaN,NaN,0,NaN,267,America/Adak,2010-01-30
17,4045412,Thirty-Seven Hill,Thirty-Seven Hill,NaN,52.84528,173.15278,T,MT,US,NaN,AK,16.0,NaN,NaN,0,NaN,193,America/Adak,2014-10-08
20,4045415,Square Point,Square Point,NaN,52.86120,173.33679,T,CAPE,US,NaN,AK,16.0,NaN,NaN,0,NaN,30,America/Adak,2014-11-17
21,4045416,Square Bluff,Square Bluff,NaN,51.65000,178.70000,T,CLF,US,NaN,AK,16.0,NaN,NaN,0,NaN,-9999,America/Adak,2014-10-08


In [12]:
print(mountains.head()[['name', 'latitude', 'longitude', 'dem','feature class']])

                 name  latitude  longitude   dem feature class
15       Vulcan Point  52.10222  177.53889 -9999             T
16     Tropical Ridge  51.99167  177.50833   267             T
17  Thirty-Seven Hill  52.84528  173.15278   193             T
20       Square Point  52.86120  173.33679    30             T
21       Square Bluff  51.65000  178.70000 -9999             T


In [13]:
geometry = gpd.points_from_xy(mountains.longitude, mountains.latitude)
gdf = gpd.GeoDataFrame(mountains, geometry=geometry, crs='EPSG:4326')
gdf.head()

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date,geometry
15,4045410,Vulcan Point,Vulcan Point,"Volcan Point,Vulcan Point",52.10222,177.53889,T,CAPE,US,NaN,AK,16.0,NaN,NaN,0,NaN,-9999,America/Adak,2014-10-08,POINT (177.53889 52.10222)
16,4045411,Tropical Ridge,Tropical Ridge,NaN,51.99167,177.50833,T,RDGE,US,NaN,AK,16.0,NaN,NaN,0,NaN,267,America/Adak,2010-01-30,POINT (177.50833 51.99167)
17,4045412,Thirty-Seven Hill,Thirty-Seven Hill,NaN,52.84528,173.15278,T,MT,US,NaN,AK,16.0,NaN,NaN,0,NaN,193,America/Adak,2014-10-08,POINT (173.15278 52.84528)
20,4045415,Square Point,Square Point,NaN,52.86120,173.33679,T,CAPE,US,NaN,AK,16.0,NaN,NaN,0,NaN,30,America/Adak,2014-11-17,POINT (173.33679 52.86120)
21,4045416,Square Bluff,Square Bluff,NaN,51.65000,178.70000,T,CLF,US,NaN,AK,16.0,NaN,NaN,0,NaN,-9999,America/Adak,2014-10-08,POINT (178.70000 51.65000)


In [14]:
print(gdf.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 225328 entries, 15 to 2237836
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   geonameid          225328 non-null  int64   
 1   name               225328 non-null  object  
 2   asciiname          225328 non-null  object  
 3   alternatenames     30933 non-null   object  
 4   latitude           225328 non-null  float64 
 5   longitude          225328 non-null  float64 
 6   feature class      225328 non-null  object  
 7   feature code       225328 non-null  object  
 8   country code       225328 non-null  object  
 9   cc2                5 non-null       object  
 10  admin1 code        225327 non-null  object  
 11  admin2 code        225170 non-null  object  
 12  admin3 code        21646 non-null   float64 
 13  admin4 code        0 non-null       float64 
 14  population         225328 non-null  int64   
 15  elevation          22441

### Working with Rasterio

In [4]:
import os
import rasterio as rio

In [6]:
data_folder = 'data'
srtm_dir = 'srtm'
filename = 'N28E087.hgt'
path = os.path.join(data_folder, srtm_dir, filename)
print(path)

data/srtm/N28E087.hgt


In [17]:
dataset = rio.open(path)

In [18]:
dataset.meta

{'driver': 'SRTMHGT',
 'dtype': 'int16',
 'nodata': -32768.0,
 'width': 3601,
 'height': 3601,
 'count': 1,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(0.0002777777777777778, 0.0, 86.99986111111112,
        0.0, -0.0002777777777777778, 29.000138888888888)}

In [19]:
# read a band from a raster
band1 = dataset.read(1)

In [20]:
print(band1)

[[5217 5211 5208 ... 5097 5098 5089]
 [5206 5201 5200 ... 5080 5075 5069]
 [5199 5194 5191 ... 5063 5055 5048]
 ...
 [5347 5345 5343 ... 5747 5750 5757]
 [5338 5338 5336 ... 5737 5740 5747]
 [5332 5331 5332 ... 5734 5736 5744]]


In [21]:
type(band1)

numpy.ndarray

In [25]:
shape = band1.shape
print(f'{shape[0]} rows x {shape[1]} cols')

3601 rows x 3601 cols


In [7]:
srtm_path = os.path.join(data_folder, srtm_dir)
all_files = os.listdir(srtm_path)

dataset_list = []

for file in all_files:
        path = os.path.join(data_folder, srtm_dir, file)
        dataset_list.append(path)
print(dataset_list)


['data/srtm/N27E086.hgt', 'data/srtm/N27E087.hgt', 'data/srtm/N28E086.hgt', 'data/srtm/N28E087.hgt']


In [27]:
from rasterio import merge
merged_result = merge.merge(dataset_list)
print(merged_result)

(array([[[4916, 4926, 4931, ..., 5097, 5098, 5089],
        [4919, 4932, 4928, ..., 5080, 5075, 5069],
        [4919, 4928, 4935, ..., 5063, 5055, 5048],
        ...,
        [ 368,  368,  366, ..., 1905, 1919, 1937],
        [ 364,  364,  362, ..., 1913, 1930, 1944],
        [ 360,  359,  357, ..., 1918, 1930, 1942]]], dtype=int16), Affine(0.0002777777777777778, 0.0, 85.99986111111112,
       0.0, -0.0002777777777777778, 29.000138888888888))


In [28]:
type(merged_result)

tuple

In [29]:
merged_data = merged_result[0]
merged_transform = merged_result[1]

In [30]:
print(merged_data.shape)

(1, 7201, 7201)


In [33]:
merged_data.reshape(-1).reshape(7201, 7201).shape

(7201, 7201)

### Working with XArray

In [2]:
import rioxarray as rxr
from rioxarray.rioxarray import xarray
from rioxarray.merge import merge_arrays

In [10]:
data_folder = 'data'
srtm_dir = 'srtm'
filename = 'N28E087.hgt'
file_path = os.path.join(data_folder, srtm_dir, filename)

In [11]:
rds = rxr.open_rasterio(file_path)

In [38]:
type(rds)

xarray.core.dataarray.DataArray

In [39]:
rds.values

array([[[5217, 5211, 5208, ..., 5097, 5098, 5089],
        [5206, 5201, 5200, ..., 5080, 5075, 5069],
        [5199, 5194, 5191, ..., 5063, 5055, 5048],
        ...,
        [5347, 5345, 5343, ..., 5747, 5750, 5757],
        [5338, 5338, 5336, ..., 5737, 5740, 5747],
        [5332, 5331, 5332, ..., 5734, 5736, 5744]]], dtype=int16)

In [40]:
type(rds.values)

numpy.ndarray

In [41]:
rds.coords

Coordinates:
  * band         (band) int64 1
  * x            (x) float64 87.0 87.0 87.0 87.0 87.0 ... 88.0 88.0 88.0 88.0
  * y            (y) float64 29.0 29.0 29.0 29.0 29.0 ... 28.0 28.0 28.0 28.0
    spatial_ref  int64 0

In [42]:
band_1 = rds.sel(band=1)
band_1

<xarray.DataArray (y: 3601, x: 3601)>
array([[5217, 5211, 5208, ..., 5097, 5098, 5089],
       [5206, 5201, 5200, ..., 5080, 5075, 5069],
       [5199, 5194, 5191, ..., 5063, 5055, 5048],
       ...,
       [5347, 5345, 5343, ..., 5747, 5750, 5757],
       [5338, 5338, 5336, ..., 5737, 5740, 5747],
       [5332, 5331, 5332, ..., 5734, 5736, 5744]], dtype=int16)
Coordinates:
    band         int64 1
  * x            (x) float64 87.0 87.0 87.0 87.0 87.0 ... 88.0 88.0 88.0 88.0
  * y            (y) float64 29.0 29.0 29.0 29.0 29.0 ... 28.0 28.0 28.0 28.0
    spatial_ref  int64 0
Attributes:
    _FillValue:    -32768.0
    scale_factor:  1.0
    add_offset:    0.0
    units:         m

In [12]:
def print_raster_properties(raster: xarray.core.dataarray.DataArray):
    print('CRS: ', rds.rio.crs)
    print('Resolution: ', rds.rio.resolution())
    print('Bounds: ', rds.rio.bounds())
    print('Width: ', rds.rio.width)
    print('Height: ', rds.rio.height)

print_raster_properties(rds)

CRS:  EPSG:4326
Resolution:  (0.0002777777777777778, -0.0002777777777777778)
Bounds:  (86.99986111111112, 27.999861111111112, 88.00013888888888, 29.000138888888888)
Width:  3601
Height:  3601


In [13]:
raster_datasets = [rxr.open_rasterio(dataset) for dataset in dataset_list]
raster_datasets

[<xarray.DataArray (band: 1, y: 3601, x: 3601)>
 [12967201 values with dtype=int16]
 Coordinates:
   * band         (band) int64 1
   * x            (x) float64 86.0 86.0 86.0 86.0 86.0 ... 87.0 87.0 87.0 87.0
   * y            (y) float64 28.0 28.0 28.0 28.0 28.0 ... 27.0 27.0 27.0 27.0
     spatial_ref  int64 0
 Attributes:
     _FillValue:    -32768.0
     scale_factor:  1.0
     add_offset:    0.0
     units:         m,
 <xarray.DataArray (band: 1, y: 3601, x: 3601)>
 [12967201 values with dtype=int16]
 Coordinates:
   * band         (band) int64 1
   * x            (x) float64 87.0 87.0 87.0 87.0 87.0 ... 88.0 88.0 88.0 88.0
   * y            (y) float64 28.0 28.0 28.0 28.0 28.0 ... 27.0 27.0 27.0 27.0
     spatial_ref  int64 0
 Attributes:
     _FillValue:    -32768.0
     scale_factor:  1.0
     add_offset:    0.0
     units:         m,
 <xarray.DataArray (band: 1, y: 3601, x: 3601)>
 [12967201 values with dtype=int16]
 Coordinates:
   * band         (band) int64 1
   * x       

In [14]:
merged = merge_arrays(raster_datasets)
merged

<xarray.DataArray (band: 1, y: 7201, x: 7201)>
array([[[4916, 4926, 4931, ..., 5097, 5098, 5089],
        [4919, 4932, 4928, ..., 5080, 5075, 5069],
        [4919, 4928, 4935, ..., 5063, 5055, 5048],
        ...,
        [ 368,  368,  366, ..., 1905, 1919, 1937],
        [ 364,  364,  362, ..., 1913, 1930, 1944],
        [ 360,  359,  357, ..., 1918, 1930, 1942]]], dtype=int16)
Coordinates:
  * x            (x) float64 86.0 86.0 86.0 86.0 86.0 ... 88.0 88.0 88.0 88.0
  * y            (y) float64 29.0 29.0 29.0 29.0 29.0 ... 27.0 27.0 27.0 27.0
  * band         (band) int64 1
    spatial_ref  int64 0
Attributes:
    _FillValue:    -32768
    scale_factor:  1.0
    add_offset:    0.0
    units:         m

In [52]:
output_file = 'output/merged_xarray_raster.tif'
merged.rio.to_raster(output_file)

### Visualizing Rasters

In [15]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
fig, axes = plt.subplots(1, 4)
fig.set_size_inches(15, 3)
plt.tight_layout()

for index, raster in enumerate(raster_datasets):
    ax = axes[index]
    raster.plot(ax=ax, cmap='Greys_r')
    ax.axis('off')
    filename = dataset_list[index]
    ax.set_title(filename)